In [0]:

confluentClusterName = "postgresql"
confluentBootstrapServers = ""
confluentTopicName = "postgres.public.basic"
confluentApiKey = ""
confluentSecret = ""
schemaRegistryUrl = ""
confluentRegistryApiKey = ""
confluentRegistrySecret = ""
deltaTablePath = '/mnt/databricks/kafka/basic'
checkpointPath = '/mnt/databricks/kafka/_checkpoint/basic'

In [0]:
from confluent_kafka.schema_registry import SchemaRegistryClient
import ssl
schema_registry_conf = {
    'url': schemaRegistryUrl,
    'basic.auth.user.info': '{}:{}'.format(confluentRegistryApiKey, confluentRegistrySecret)}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [0]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType
binary_to_string = fn.udf(lambda x: str(int.from_bytes(x, byteorder='big')), StringType())
streamTestDf = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", confluentBootstrapServers)
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", confluentTopicName)
  .option("startingOffsets", "earliest")
  .option("failOnDataLoss", "false")
  .load()
  .filter(fn.col("value").isNotNull())
  .withColumn('key', fn.col("key").cast(StringType()))
  .withColumn('fixedValue', fn.expr("substring(value, 6, length(value)-5)"))
  .withColumn('valueSchemaId', binary_to_string(fn.expr("substring(value, 2, 4)")))
  .select('topic', 'partition', 'offset', 'timestamp', 'timestampType', 'key', 'valueSchemaId','fixedValue')
)


In [0]:
display(streamTestDf) 

topic,partition,offset,timestamp,timestampType,key,valueSchemaId,fixedValue
postgres.public.basic,0,0,2024-05-16T09:05:38.545Z,0,"{""id"":1}",100007,AAICAgphcHBsZRYyLjQuMi5GaW5hbBRwb3N0Z3Jlc3FsEHBvc3RncmVznOGxivBjAApmaXJzdBJrYWZrYV9jZGMCIltudWxsLCI2NzIxMzg5NiJdDHB1YmxpYwpiYXNpYwKIDwKQ6YxAAAJyAqDwsorwYwA=
postgres.public.basic,0,1,2024-05-16T09:05:38.546Z,0,"{""id"":2}",100007,AAIEAgxiYW5hbmEWMi40LjIuRmluYWwUcG9zdGdyZXNxbBBwb3N0Z3Jlc5zhsYrwYwAIbGFzdBJrYWZrYV9jZGMCIltudWxsLCI2NzIxMzg5NiJdDHB1YmxpYwpiYXNpYwKIDwKQ6YxAAAJyAqTwsorwYwA=
postgres.public.basic,0,2,2024-05-16T09:09:04.392Z,0,"{""id"":3}",100007,AAIGAhBtYW5kYXJpbhYyLjQuMi5GaW5hbBRwb3N0Z3Jlc3FsEHBvc3RncmVzgunLivBjAApmYWxzZRJrYWZrYV9jZGMCIltudWxsLCI4Mzg4Nzk4NCJdDHB1YmxpYwpiYXNpYwLADwLgnYBQAAJjAuyTzIrwYwA=
postgres.public.basic,0,3,2024-05-16T09:09:04.392Z,0,"{""id"":4}",100007,AAIIAhJwaW5lYXBwbGUWMi40LjIuRmluYWwUcG9zdGdyZXNxbBBwb3N0Z3Jlc7jpy4rwYwAKZmFsc2USa2Fma2FfY2RjAi5bIjgzODg4MzUyIiwiODM4ODgzNTIiXQxwdWJsaWMKYmFzaWMCwg8CwKOAUAACYwLuk8yK8GMA
postgres.public.basic,0,4,2024-05-16T09:09:53.687Z,0,"{""id"":2}",100007,AgQAABYyLjQuMi5GaW5hbBRwb3N0Z3Jlc3FsEHBvc3RncmVzsJHSivBjAApmYWxzZRJrYWZrYV9jZGMCLlsiODM4ODg1MzYiLCI4Mzg5MDEwNCJdDHB1YmxpYwpiYXNpYwLSDwLwvoBQAAJkAvCV0orwYwA=
postgres.public.basic,0,6,2024-05-16T09:11:18.62Z,0,"{""id"":5}",100007,AAIKAhJyYXNwYmVycnkWMi40LjIuRmluYWwUcG9zdGdyZXNxbBBwb3N0Z3Jlc/K23IrwYwAKZmFsc2USa2Fma2FfY2RjAi5bIjgzODkwMjE2IiwiODM4OTI1NTIiXQxwdWJsaWMKYmFzaWMC6A8CkOWAUAACYwLmvdyK8GMA
postgres.public.basic,0,7,2024-05-16T09:11:18.62Z,0,"{""id"":6}",100007,AAIMAg5tb3JhbmdvFjIuNC4yLkZpbmFsFHBvc3RncmVzcWwQcG9zdGdyZXOot9yK8GMACmZhbHNlEmthZmthX2NkYwIuWyI4Mzg5MjczNiIsIjgzODkyNzM2Il0McHVibGljCmJhc2ljAuoPAoDogFAAAmMC6L3civBjAA==
postgres.public.basic,0,8,2024-05-16T09:11:51.58Z,0,"{""id"":6}",100007,AAIMAhRzdHJhd2JlcnJ5FjIuNC4yLkZpbmFsFHBvc3RncmVzcWwQcG9zdGdyZXP6xeCK8GMACmZhbHNlEmthZmthX2NkYwIuWyI4Mzg5MjkyMCIsIjgzODk0MDgwIl0McHVibGljCmJhc2ljAvgPAoD9gFAAAnUCpsfgivBjAA==
postgres.public.basic,0,9,2024-05-16T09:13:25.435Z,0,"{""id"":5}",100007,AgoCEnJhc3BiZXJyeQIKAhJmcmFtYm9lc2EWMi40LjIuRmluYWwUcG9zdGdyZXNxbBBwb3N0Z3Jlc+Lx64rwYwAKZmFsc2USa2Fma2FfY2RjAjBbIjgzODk0MjE2IiwiMTAwNjYzNzkyIl0McHVibGljCmJhc2ljApIQAuCHgGA= (truncated)


In [0]:
import pyspark.sql.functions as fn
from pyspark.sql.avro.functions import from_avro
def parseAvroDataWithSchemaId(df, ephoch_id):
  cachedDf = df.cache()
  fromAvroOptions = {"mode":"FAILFAST"}
  def getSchema(id):
    return str(schema_registry_client.get_schema(id).schema_str)
  distinctValueSchemaIdDF = cachedDf.select(fn.col('valueSchemaId').cast('integer')).distinct()
  for valueRow in distinctValueSchemaIdDF.collect():
    currentValueSchemaId = sc.broadcast(valueRow.valueSchemaId)
    currentValueSchema = sc.broadcast(getSchema(currentValueSchemaId.value))
    filterValueDF = cachedDf.filter(fn.col('valueSchemaId') == currentValueSchemaId.value)
    filterValueDF \
      .select('topic', 'partition', 'offset', 'timestamp', 'timestampType', 'key', from_avro('fixedValue', currentValueSchema.value, fromAvroOptions).alias('parsedValue')) \
      .write \
      .format("delta") \
      .mode("append") \
      .option("mergeSchema", "true") \
     .save(deltaTablePath)


In [0]:
streamTestDf.writeStream \
  .option("checkpointLocation", checkpointPath) \
  .foreachBatch(parseAvroDataWithSchemaId) \
  .queryName("TestCDCPostgreSQLDebeziumConfluent") \
  .start()


In [0]:
deltaTestDf = spark.read.format("delta").load(deltaTablePath)
display(deltaTestDf)


topic,partition,offset,timestamp,timestampType,key,parsedValue
postgres.public.basic,0,0,2024-05-16T09:05:38.545Z,0,"{""id"":1}","List(null, List(1, apple), List(2.4.2.Final, postgresql, postgres, 1715850328142, first, kafka_cdc, [null,""67213896""], public, basic, 964, 67213896, null), r, 1715850337296, null)"
postgres.public.basic,0,1,2024-05-16T09:05:38.546Z,0,"{""id"":2}","List(null, List(2, banana), List(2.4.2.Final, postgresql, postgres, 1715850328142, last, kafka_cdc, [null,""67213896""], public, basic, 964, 67213896, null), r, 1715850337298, null)"
postgres.public.basic,0,2,2024-05-16T09:09:04.392Z,0,"{""id"":3}","List(null, List(3, mandarin), List(2.4.2.Final, postgresql, postgres, 1715850541633, false, kafka_cdc, [null,""83887984""], public, basic, 992, 83887984, null), c, 1715850544374, null)"
postgres.public.basic,0,3,2024-05-16T09:09:04.392Z,0,"{""id"":4}","List(null, List(4, pineapple), List(2.4.2.Final, postgresql, postgres, 1715850541660, false, kafka_cdc, [""83888352"",""83888352""], public, basic, 993, 83888352, null), c, 1715850544375, null)"
postgres.public.basic,0,4,2024-05-16T09:09:53.687Z,0,"{""id"":2}","List(List(2, null), null, List(2.4.2.Final, postgresql, postgres, 1715850593368, false, kafka_cdc, [""83888536"",""83890104""], public, basic, 1001, 83890104, null), d, 1715850593656, null)"
postgres.public.basic,0,6,2024-05-16T09:11:18.62Z,0,"{""id"":5}","List(null, List(5, raspberry), List(2.4.2.Final, postgresql, postgres, 1715850677689, false, kafka_cdc, [""83890216"",""83892552""], public, basic, 1012, 83892552, null), c, 1715850678131, null)"
postgres.public.basic,0,7,2024-05-16T09:11:18.62Z,0,"{""id"":6}","List(null, List(6, morango), List(2.4.2.Final, postgresql, postgres, 1715850677716, false, kafka_cdc, [""83892736"",""83892736""], public, basic, 1013, 83892736, null), c, 1715850678132, null)"
postgres.public.basic,0,8,2024-05-16T09:11:51.58Z,0,"{""id"":6}","List(null, List(6, strawberry), List(2.4.2.Final, postgresql, postgres, 1715850711421, false, kafka_cdc, [""83892920"",""83894080""], public, basic, 1020, 83894080, null), u, 1715850711507, null)"
postgres.public.basic,0,9,2024-05-16T09:13:25.435Z,0,"{""id"":5}","List(List(5, raspberry), List(5, framboesa), List(2.4.2.Final, postgresql, postgres, 1715850804337, false, kafka_cdc, [""83894216"",""100663792""], public, basic, 1033, 100663792, null), u, 1715850804938, null)"


In [0]:
deltaTestDf.count()

9

In [0]:
deltaTestDf.printSchema()

root
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- key: string (nullable = true)
 |-- parsedValue: struct (nullable = true)
 |    |-- before: struct (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- fruit: string (nullable = true)
 |    |-- after: struct (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- fruit: string (nullable = true)
 |    |-- source: struct (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |    |-- connector: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- ts_ms: long (nullable = true)
 |    |    |-- snapshot: string (nullable = true)
 |    |    |-- db: string (nullable = true)
 |    |    |-- sequence: string (nullable = true)
 |    |    |-- schema: string (nullable = true)
 |    |    |-- ta

In [0]:
new_df = (deltaTestDf
          .withColumn('id', fn.col('parsedValue.after.id'))
          .withColumn('fruit', fn.col('parsedValue.after.fruit'))
          .withColumn('operation',  fn.when(fn.col('parsedValue.op')=='c', 'insert')
                                      .when(fn.col('parsedValue.op')=='r', 'read')
                                      .when(fn.col('parsedValue.op')=='u', 'update')
                                      .when(fn.col('parsedValue.op')=='d', 'delete')
                      )
        #   .filter(fn.col('id').isNotNull())
          .select(['topic','partition','offset','timestamp','timestampType','key','id','fruit','operation'])
    
    )

# Now 'new_df' contains exploded columns from 'parsedValue'
new_df.display()

topic,partition,offset,timestamp,timestampType,key,id,fruit,operation
postgres.public.basic,0,0,2024-05-16T09:05:38.545Z,0,"{""id"":1}",1,apple,read
postgres.public.basic,0,1,2024-05-16T09:05:38.546Z,0,"{""id"":2}",2,banana,read
postgres.public.basic,0,2,2024-05-16T09:09:04.392Z,0,"{""id"":3}",3,mandarin,insert
postgres.public.basic,0,3,2024-05-16T09:09:04.392Z,0,"{""id"":4}",4,pineapple,insert
postgres.public.basic,0,4,2024-05-16T09:09:53.687Z,0,"{""id"":2}",null,null,delete
postgres.public.basic,0,6,2024-05-16T09:11:18.62Z,0,"{""id"":5}",5,raspberry,insert
postgres.public.basic,0,7,2024-05-16T09:11:18.62Z,0,"{""id"":6}",6,morango,insert
postgres.public.basic,0,8,2024-05-16T09:11:51.58Z,0,"{""id"":6}",6,strawberry,update
postgres.public.basic,0,9,2024-05-16T09:13:25.435Z,0,"{""id"":5}",5,framboesa,update
